#Introduction to Machine Learning  
**Computer Vision Hackathon  
Wintersession  
Tuesday, January 24, 2023**

In this notebook you will create a CNN from scratch to distinguish cats from dogs.

# About Your Colab Session

Learn about the CPU-cores for your session:

In [ ]:
cat /proc/cpuinfo

In [ ]:
import os
num_cores = min(os.cpu_count(), 2)
print(num_cores)

Let's see which GPU we are using (probably a Tesla T4):

In [ ]:
!nvidia-smi

# Data Preparation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from PIL import Image

We want to use a GPU when one is available:

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
torch.manual_seed(42)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs  = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': num_cores, 'pin_memory': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

Download and unpack the data:

In [ ]:
!wget https://tigress-web.princeton.edu/~jdh4/cats_vs_dogs.tar
!tar xf cats_vs_dogs.tar

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
]) 
train_set = datasets.ImageFolder(root="./training_set/", transform=transform)
test_set = datasets.ImageFolder(root="./test_set/", transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **train_kwargs)
test_loader  = torch.utils.data.DataLoader(test_set, shuffle=True, **test_kwargs)

image_0, label_0 = train_set[0]
print(image_0.shape, label_0)

There are roughly 4000 cat images and 4000 dog images in the training set. The test set is roughly 1000 images of each. All images have dimensions 128x128. The cat and dogs images are in color so they are composed of three layers (red, green, blue). The MNIST data set was grayscale so only a single layer was needed per image.

In [ ]:
img = Image.open("./training_set/dogs/resized-dog.1001.jpg")
print(f"Image height: {img.height}") 
print(f"Image width: {img.width}")
img

In [ ]:
img = Image.open("./training_set/cats/resized-cat.1001.jpg")
print(f"Image height: {img.height}") 
print(f"Image width: {img.width}")
img

# Model Definition and Hackathon Project

The hackathon project is to create a convolutional neural network from scratch and train in such that it gives a sustained accuracy of 75% or higher on the test set. Your network should use at least 3 convolutional layers.

You only need to write the Net class. The rest of the notebook does not need to be changed. After writing the Net class, try running notebook. Raise your hand if you have any questions for the instructor. We're happy to give hints as you work through the exercise.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CREATE THE LAYERS HERE

    def forward(self, x):
        # DEFINE THE FORWARD PASS HERE
        return output

Instantiate the network and move it to the device (which is a GPU when available). Create the optimizer.

In [ ]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 128, 128))

# Train and Test Methods

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train() # sets the model in training mode (i.e., dropout enabled)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    model.eval() # sets the model in evaluation mode (i.e., dropout disabled)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train for some number of epochs while reporting the accuracy on the test set periodically:

In [ ]:
epochs = 12
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()